In [101]:
import re
import os
import numpy as np
import copy

In [112]:
# Defining Helper variables to be used later

labelList = ["O", "B-positive", "B-negative", "B-neutral", "I-positive", "I-negative", "I-neutral"]
labelDictInit = {   
        "START": 0,
        "O": 0,
        "B-positive": 0,
        "B-negative": 0,
        "B-neutral": 0,
        "I-positive": 0,
        "I-negative": 0,
        "I-neutral": 0,
        "END": 0
    }

NUMBER_OF_LABELS = len(labelList)

# Initialise a random number generator with a fixed seed for reproducible results and deterministic behavior
rng = np.random.default_rng(1004519 + 1004103 + 1004555)

# Defining the filePath for the datasets
folderPath = os.path.abspath(os.getcwd())

EsTrainFilePath = os.path.join(folderPath, "../Data/ES/train")
EsDevInFilePath = os.path.join(folderPath, "../Data/ES/dev.in")
EsDevOutFilePath = os.path.join(folderPath, "../Data/ES/dev.out")
EsPredOutputFilePath = os.path.join(folderPath, "../Data/ES/dev.p1.out")

RuTrainFilePath = os.path.join(folderPath, "../Data/RU/train")
RuDevInFilePath = os.path.join(folderPath, "../Data/RU/dev.in")
RuDevOutFilePath = os.path.join(folderPath, "../Data/RU/dev.out")
RuPredOutputFilePath = os.path.join(folderPath, "../Data/RU/dev.p1.out")

ES_FilePaths = [
    EsTrainFilePath,
    labelDictInit,
    EsDevInFilePath,
    EsDevOutFilePath,
    EsPredOutputFilePath]

RU_FilePaths = [
    RuTrainFilePath,
    labelDictInit,
    RuDevInFilePath,
    RuDevOutFilePath,
    RuPredOutputFilePath]

languages = [ES_FilePaths, RU_FilePaths]


In [103]:
# Helper functions to read and parse data
def readFile(filePath: str):
    with open(filePath, "r", encoding="utf-8") as f:
        return f.readlines()
    
def processFile(file: list):
    return [word[:len(word)-1] for word in file]

def getAllUniqueTokens(input_data):
    # Might want to somehow ensure that this order stays consistent between runs
    return list(set(item.split()[0] for item in input_data if item))

### Part 1

1. Write a function that estimates the emission parameters from the training set using MLE (maximum likelihood estimation):
<br>
$$
e(x|y) = \frac{{\text{{Count}}(y \rightarrow x)}}{{\text{{Count}}(y)}}
$$

2. Set k to 1, implement this fix into your function for computing the emission parameters

$$
e(x|y) = \begin{cases}
\frac{{\text{{Count}}(y \rightarrow x)}}{{\text{{Count}}(y)+k}}, & \text{{if the word token }} x \text{{ appears in the training set}} \\
\frac{k}{{\text{{Count}}(y)+k}}, & \text{{if word token }} x \text{{ is the special token \#UNK\#}}
\end{cases}




In [120]:
# Calculating Emissions Function

def calcCountofEachWord(file: list, labelDict_in: dict):
    tokenDict_out = {}

    for i in range(len(file)):
        if file[i] != "":
            l = file[i].split()
            token = l[0]
            label = l[1]
            key = (token, label)
            if label in labelDict_in:
                labelDict_in[label] += 1

            else:
                labelDict_in[label] = 1
                

            if key in tokenDict_out:
                tokenDict_out[key] += 1
            else:
                tokenDict_out[key] = 1
    return tokenDict_out, labelDict_in

def calcEmission(tokenDict_in: dict, labelDict_in: dict, uniqueTokensList_in: list, k: float = 1.0):
    emissionDict_out = {}
    unknownDict = {}

    for token, label in tokenDict_in.keys():
        inner_dict = {}
        for key, val in labelDict_in.items():
            if val != 0 and label == key:
                e = tokenDict_in[(token, label)] / (val + k)
                inner_dict[label] = e
                                
        emissionDict_out[token] = inner_dict

    for key, val in labelDict_in.items():
        # Creating entry for unknown words
        unknownToken = "#UNK"
        e = k / (val + k)
        unknownDict[unknownToken] = e
        emissionDict_out[unknownToken] = unknownDict

    
    return emissionDict_out

def getLabel(tokenInput: str, uniqueTokensList_in: list, emissionsDict_in: dict):
    if tokenInput in uniqueTokensList_in:
        return max(emissionsDict_in[tokenInput], key=emissionsDict_in[tokenInput].get)
    else:
        return max(emissionsDict_in["#UNK"], key=emissionsDict_in["#UNK"].get)
        
    ...

def calcSentimentAnalysis(tokenList: list, trainedData: dict, uniqueTokensList_in: list):

    predictedTokenList = []

    for token in tokenList:
        if token:
            if token in trainedData:
                predictedTokenList.append(token + " " + getLabel(token, uniqueTokensList_in, trainedData))
            else:
                predictedTokenList.append(token + " " + getLabel(token, uniqueTokensList_in, trainedData))
        else:
            predictedTokenList.append("")
    return predictedTokenList


def evalModel(predictedTokenFilePath: str, actualTokenFilePath: str):
    
    # Reading both files
    predictedTokenFile = processFile(readFile(predictedTokenFilePath))
    actualTokenFile = processFile(readFile(actualTokenFilePath))

    print("\n=============Processed Files=============")
    print(f"Predicted Token File: {predictedTokenFile}")
    print(f"Actual Token File: {actualTokenFile}")
    print("=============Processed Files=============\n")

    # Extracting the predicted and actual labels
    predictedLabels = [line.split()[1] for line in predictedTokenFile if line]
    actualLabels = [line.split()[1] for line in actualTokenFile if line]

    print("\n=============Label Lists=============")
    print(f"Predicted Labels: {predictedLabels}")
    print(f"Actual Labels: {actualLabels}")
    print("=============Label Lists=============\n")
    
    # Checking if the shape of the label lists are the same
    assert len(predictedLabels) == len(actualLabels) 
    
    # Precision
    correctPredictions = 0
    # Calculate metrics for each label
    totalPredicted = len(predictedLabels)
    totalActual = len(actualLabels)

    for i in range(totalPredicted):    
        if predictedLabels[i] == actualLabels[i]:
            correctPredictions += 1
    precision = round(correctPredictions/len(predictedLabels), 5) if (totalPredicted) != 0 else 0

    # Recall
    recall = round(correctPredictions/len(actualLabels), 5) if (totalActual) != 0 else 0

    # F1
    f1 = round(2 * ((precision * recall) / (precision + recall)), 5) if ((totalPredicted) != 0 ) and ((totalActual) != 0) else 0

    print("\n=============Evaluation Metrics=============")
    print("Precision: ", precision)
    print("Recall: ", recall)
    print("F1: ", f1)
    print("=============Evaluation Metrics=============")

    return precision, recall, f1
    
    

In [105]:
# Calling the functions
emissionsDict = {}

def predictAndWrite(
        trainFilePath: str, 
        labelDictIn: dict, 
        devInFilePath: str, 
        devOutFilePath: str,
        predOutputFilePath: str,
    ):
    # Processing the file to separate line by line
    trainData = processFile(readFile(filePath=trainFilePath))
    uniqueTokensList = getAllUniqueTokens(trainData)

    # Calculating the count of each token to the label
    tokenDict, labelDictIn = calcCountofEachWord(trainData, labelDictIn)

    # Calculating the emission value for each unique token

    emissionsDict = calcEmission(tokenDict, labelDictIn, uniqueTokensList)

    # Reading test file
    testInData = processFile(readFile(devInFilePath))

    # Predicting test data
    predictedLabels = calcSentimentAnalysis(testInData, emissionsDict, uniqueTokensList)

    # Writing to file
    with open(predOutputFilePath, "w+", encoding="utf-8") as file:
        for line in predictedLabels:
            file.write(line + "\n")

    # Calculating Precision
    precision, recall, f1 = evalModel(predOutputFilePath, devOutFilePath)

    return precision, recall, f1


In [121]:
# Running the function
for language in languages:
    predictAndWrite(
        language[0],
        language[1],
        language[2],
        language[3],
        language[4]
    )


=============Processed Files=============
Predicted Token File: ['Plato B-negative', 'degustación I-negative', ': O', 'un O', 'poco O', 'abundante O', 'de I-negative', 'más O', ', I-negative', 'pero B-positive', 'bien B-positive', 'cocinado O', '. I-positive', '', 'restaurante O', 'excelente O', 'con I-negative', 'carne B-positive', 'de I-negative', 'alta O', 'calidad B-negative', '. I-positive', '', 'Las O', 'posibilidades #UNK', 'en I-negative', 'el I-neutral', 'restaurante O', 'son O', 'fundamentalmente #UNK', 'tres I-negative', '; O', 'carta B-neutral', 'normal O', ', I-negative', 'menú B-negative', 'degustacion O', 'y I-positive', 'una O', 'opción O', 'intermedia #UNK', 'que O', 'es I-positive', 'una O', 'selección O', 'de I-negative', 'primeros O', 'y I-positive', 'postres B-neutral', 'y I-positive', 'carta B-neutral', 'para I-negative', 'el I-neutral', 'segundo B-negative', '. I-positive', '', 'No I-positive', 'perderse #UNK', 'el I-neutral', 'sorbete B-positive', 'de I-negativ